In [2]:
import pandas as pd
import llama_index
from llama_index.llms import OpenAI
import json

In [3]:
llm = OpenAI("gpt-4-1106-preview")


In [4]:
inferred_data = pd.read_csv("Gurukul Data - data_inferred.csv")
inferred_data.head(1)

,Category,ID,Question,Question Context,Scripture,Split,prompt_openai,prompt_str,used_context,gpt3_responses,gpt4_response,Unnamed: 11
0,From Context,1,Why does Vasishta refuse to give Sabala to Vis...,Context 1\nहैरण्यानां रथानां च श्वेताश्वानां च...,Ramayana,Train,"[{""role"": ""system"", ""content"": ""You are an exp...",system: You are an expert Q&A system that is t...,"[""sarga: 54\nfile_path: data/ramayana/data/bal...",Vasishta refuses to give Sabala to Viswamitra ...,Vasishta refuses to give Sabala to Viswamitra ...,NaN


In [48]:
REFINE_PROMPT="""You are an critique for a QnA app called "Gurukul" which uses scriptures to answer spiritual questions.

The app was asked to follow these rules.

1. Be respectful to Hindu dharma followers; avoid potential offense.
2. Rely solely on provided context, not prior knowledge.
3. Directly answer; omit phrases like 'Based on the context...'.
4. Quote sources, chapters, kandas, sargas, and shlokas, explaining their relevance.
5. Format using markdowns and emojis, e.g., blockquote for Sanskrit shlokas.
6. Add a disclaimer: Answers may be subjective; research independently.

The format should be: 
"<Brief answer>
<Explanations with sources quoted>
<Final Summary/ Conclusion>"

## Provided Contexts

{context_str}

## Query form user

{query_str}

## Response from Gurukul app

{response_str}


## Critique  + Better response

Now critique the answer provided by Gurukul and find out the mistakes if any. If the answer is not satisfactory, modify the response and print it.
"""

def get_refine_prompt(row):
    context_str = "\n\n".join(json.loads(row.used_context))
    return REFINE_PROMPT.format(context_str = context_str, query_str = row.Question, response_str = row.gpt4_response)

In [50]:
row = inferred_data.iloc[52]
prompt = get_refine_prompt(row)
print(prompt)

You are an critique for a QnA app called "Gurukul" which uses scriptures to answer spiritual questions.

The app was asked to follow these rules.

1. Be respectful to Hindu dharma followers; avoid potential offense.
2. Rely solely on provided context, not prior knowledge.
3. Directly answer; omit phrases like 'Based on the context...'.
4. Quote sources, chapters, kandas, sargas, and shlokas, explaining their relevance.
5. Format using markdowns and emojis, e.g., blockquote for Sanskrit shlokas.
6. Add a disclaimer: Answers may be subjective; research independently.

The format should be: 
"<Brief answer>
<Explanations with sources quoted>
<Final Summary/ Conclusion>"

## Provided Contexts

sarga: 11
file_path: data/ramayana/data/aranyakanda.csv
kanda: aranya kanda

अस्माकं कस्यचित् स्थानमेष प्रार्थयते मुनिः।।3.11.14।।इति संविग्नमनसस्सर्वे ते त्रिदिवौकसः।
  'The gods of heaven became agitated in their minds thinking the sage might usurp one of their places.

तत्र कर्तुं तपोविघ्नं देवैस्

In [15]:
# llm.predict()